In [26]:
import pandas as pd
import numpy as np
import re
import pickle as pkl
import time

In [27]:
ori_pred_name = "../output/train_pred.v2.csv"

In [4]:
ori_df = pd.read_csv(ori_pred_name)
ori_df.shape

(9918441, 4)

In [7]:
ori_diff_df = ori_df.loc[ori_df["true"] != ori_df["class_pred"]]
ori_diff_df.shape

(6415, 4)

In [13]:
ori_diff_df.columns

Index(['after', 'before', 'true', 'class_pred'], dtype='object')

In [16]:
ori_diff_df["true"].value_counts()

LETTERS       3460
PLAIN         1185
CARDINAL       912
DIGIT          338
DATE           265
DECIMAL        136
VERBATIM        84
PUNCT           15
ELECTRONIC      10
ORDINAL          5
TELEPHONE        1
TIME             1
ADDRESS          1
FRACTION         1
MONEY            1
Name: true, dtype: int64

In [19]:
ori_diff_df.loc[ori_diff_df["true"] == "LETTERS"]

,after,before,true,class_pred
1805,p o m z,POMZ,LETTERS,PLAIN
1822,b o e v,Boev,LETTERS,PLAIN
4122,p i a l a t,PIALAT,LETTERS,PLAIN
12713,j o e p,Joep,LETTERS,PLAIN
17622,e e m a,Eema,LETTERS,PLAIN
18958,t a's,Tas,LETTERS,PLAIN
23658,c e n s o s,CENSOS,LETTERS,PLAIN
27751,u b o's,Ubos,LETTERS,PLAIN
28340,p o a,Poa,LETTERS,PLAIN
30365,r o d t,Rodt,LETTERS,PLAIN


使用patch

### patch方案一

In [31]:
with open("../output/letter_set.pkl", "rb") as f:
    letter_set = pkl.load(f)


def is_com(token):
    token = str(token)
    pt = re.compile(r"\.[a-zA-Z]{2,}")
    return True if pt.search(token) else False


def is_year(token):
    token = str(token)
    return token.isdigit() and len(token) == 4 and 1000 <= int(token) <= 3000


In [32]:
pred_df = ori_df.copy()
count = 0
for i, row in pred_df.iterrows():
    # 1.被误认为是cardinal的实际上的date的年份
    if row["class_pred"] == "CARDINAL":
        if is_year(row["before"]):
            # 不能直接修改row，因为row是副本
            pred_df.loc[i, "class_pred"] = "DATE"
            count += 1
pred_df.loc[pred_df["true"] != pred_df["class_pred"]].shape[0]

In [34]:
pred_df = ori_df.copy()
count = 0
for i, row in pred_df.iterrows():
    # 1.被误认为是cardinal的实际上的date的年份
    if row["class_pred"] == "CARDINAL":
        if is_year(row["before"]) and pred_df.loc[i + 1, "before"] not in ["-", "/"]:
            pred_df.loc[i, "class_pred"] = "DATE"
            count += 1
pred_df.loc[pred_df["true"] != pred_df["class_pred"]].shape[0]

7685

In [ ]:
count = 0
for i, row in pred_df.iterrows():
    # 1.被误认为是cardinal的实际上的date的年份
    if row["class_pred"] == "CARDINAL":
        if is_year(row["before"]):
#         if is_year(row["before"]) and i + 1 < data_num and pred_df.loc[i + 1, "before"] not in ["-", "/"]:
            # 不能直接修改row，因为row是副本
            pred_df.loc[i, "class_pred"] = "DATE"
            count += 1
    elif row["class_pred"] == "PLAIN":
        # 2.被误认为是plain的实际上的electronic的网站
        if is_com(row["before"]):
            pred_df.loc[i, "class_pred"] = "ELECTRONIC"
            count += 1
        # 3.被误认为是plain的实际上是letters
        elif row["before"] in letter_set:
            pred_df.loc[i, "class_pred"] = "LETTERS"
            count += 1
    elif row["class_pred"] == "LETTERS":
        if row["before"] not in letter_set:
            pred_df.loc[i, "class_pred"] = "PLAIN"
            count += 1

In [8]:
fixed_diff_df = fixed_df.loc[fixed_df["true"] != fixed_df["class_pred"]]
fixed_diff_df.shape

(12635, 4)

In [20]:
fixed_diff_df["true"].value_counts()

LETTERS      9565
CARDINAL     2214
DIGIT         338
DATE          233
DECIMAL       136
VERBATIM       84
PLAIN          40
PUNCT          15
ORDINAL         5
TELEPHONE       1
ADDRESS         1
MONEY           1
TIME            1
FRACTION        1
Name: true, dtype: int64

In [22]:
fixed_diff_df.loc[fixed_diff_df["true"] == "LETTERS"]

,after,before,true,class_pred
2476,M,M,LETTERS,PLAIN
4043,F,F,LETTERS,PLAIN
4045,X,X,LETTERS,PLAIN
4347,b,b,LETTERS,PLAIN
5422,T,T,LETTERS,PLAIN
15026,m,m,LETTERS,PLAIN
15117,B,B,LETTERS,PLAIN
15122,T,T,LETTERS,PLAIN
16309,C,C,LETTERS,PLAIN
16311,C,C,LETTERS,PLAIN
